In [162]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\diego\AppData\Local\Temp\ipykernel_13128\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [163]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [164]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn  as sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [165]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [166]:
#Your Code:
print(data.head())

                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0


In [167]:
#You Code here:
from sklearn.model_selection import train_test_split
data_train, data_val = train_test_split(data, test_size=0.3, random_state=42, stratify=data['label'])

## Data Preprocessing

pip install nltk

In [168]:
#First we need to import some libraries first:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [169]:
# Your code
from bs4 import BeautifulSoup
import re

#Sample HTML content
html_content = """
<html>
    <body>
        <p>Hello, this is a <b>test</b> message with some <a href="#">HTML</a> content.</p>
        <p>This includes unnecessary words like 'remove' and 'clean'.</p>
    </body>
</html>
"""
# Step 1: Remove HTML Tags

soup = BeautifulSoup(html_content, "html.parser")
cleaned_text = soup.get_text()

# Step 2: Define the list of unwanted words/phrases to remove
unwanted_words = ["remove", "clean"]

# Step 3: Remove unwanted words using regex
pattern = re.compile(r'\b(?:' + '|'.join(unwanted_words) + r')\b', re.IGNORECASE)
cleaned_text = pattern.sub('', cleaned_text)

# Step 4: Optional - Remove extra spaces left after word removal
cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

# Print the cleaned content

print(cleaned_text)

Hello, this is a test message with some HTML content. This includes unnecessary words like '' and ''.


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [170]:
# Your code
from bs4 import BeautifulSoup
import re

#Funtion to remove Html Content and unwanted words
def clean_html_and_remove_words(html_content, unwanted_words):
    #Remove Tags
    soup = BeautifulSoup(html_content, "html.parser")
    cleaned_text = soup.get_text()
    #Remove unwanted words
    pattern = re.compile(r'\b(?:' + '|'.join(unwanted_words) + r')\b', re.IGNORECASE)
    cleaned_text = pattern.sub('', cleaned_text)
    #Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

#Define the list of unwanted words/phrases to remove
unwanted_words = ["remove", "clean", "unnecessary"]

#Define a function that will apply the cleaning process to a column of text
data['cleaned_text'] = data['text'].apply(lambda x: clean_html_and_remove_words(x, unwanted_words))

#Print the cleaned content
print(data[['text', 'cleaned_text']].head())

C:\Users\diego\AppData\Local\Temp\ipykernel_13128\69838825.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, "html.parser")
C:\Users\diego\AppData\Local\Temp\ipykernel_13128\69838825.py:8: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_content, "html.parser")


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...  
1                                           Will do.  
2  Nora--Cheryl has emailed dozens of memos about...  
3  Dear Sir=2FMadam=2C I know that this proposal ...  
4                                                fyi  


## Now let's work on removing stopwords
Remove the stopwords.

In [171]:
# Your code
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords') 

#Get list of stop words in English
stop_wrods = set(stopwords.words('english'))

#Function to remove stopwords from text
def remove_stopwords(text):
    words = text.split()
    filterd_words = [word for word in words if word not in stop_wrods]
    return ' '.join(filterd_words)

#Apply the function to the clean text column
data['no_stopwords_text'] = data['cleaned_text'].apply(remove_stopwords)

#Show the original and the cleaned text and stopwords-free text
print(data[['text', 'cleaned_text', 'no_stopwords_text']].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                   no_stopwords_text  
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...  
1                                           Will do.  
2  Nora--Cheryl emailed dozens memos Haiti weeken...  
3  Dear Sir=2FMadam=2C I know proposal might surp...  
4                                                fy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

pip install --upgrade spacy typing-extensions

In [172]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download necessary NLTK datasets (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization on text
def lemmatize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the lemmatized words back into a single string
    return ' '.join(lemmatized_words)

# Apply the lemmatization function to the 'no_stopwords_text' column
data['lemmatized_text'] = data['no_stopwords_text'].apply(lemmatize_text)

# Show the original, cleaned, stopword-free, and lemmatized text
print(data[['text', 'cleaned_text', 'no_stopwords_text', 'lemmatized_text']].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                   no_stopwords_text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl emailed dozens memos Haiti weeken...   
3  Dear Sir=2FMadam=2C I know proposal might surp...   
4                                             

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [173]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Separate spam and ham messages
ham_messages = data[data['label'] == 0]['lemmatized_text']
spam_messages = data[data['label'] == 1]['lemmatized_text']

# Initialize separate CountVectorizers for ham and spam
ham_vectorizer = CountVectorizer(stop_words='english')
spam_vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizers on ham and spam messages separately and get word counts
ham_word_counts = ham_vectorizer.fit_transform(ham_messages)
spam_word_counts = spam_vectorizer.fit_transform(spam_messages)

# Sum up the word counts
ham_word_freq = ham_word_counts.sum(axis=0).A1
spam_word_freq = spam_word_counts.sum(axis=0).A1

# Get words from the vectorizer vocabulary
ham_vocab = ham_vectorizer.get_feature_names_out()
spam_vocab = spam_vectorizer.get_feature_names_out()

# Create DataFrames for easier analysis
ham_freq_df = pd.DataFrame({'word': ham_vocab, 'frequency': ham_word_freq}).sort_values(by='frequency', ascending=False)
spam_freq_df = pd.DataFrame({'word': spam_vocab, 'frequency': spam_word_freq}).sort_values(by='frequency', ascending=False)

# Get the top 10 words for ham and spam
top_10_ham = ham_freq_df.head(10)
top_10_spam = spam_freq_df.head(10)

# Display the top 10 words for ham and spam
print("Top 10 Words in Ham Messages:")
print(top_10_ham)

print("\nTop 10 Words in Spam Messages:")
print(top_10_spam)


Top 10 Words in Ham Messages:
           word  frequency
4896         pm        113
6044      state        110
5029  president         99
4266         mr         86
4477      obama         84
153        2010         82
243          30         81
6481       time         80
4812    percent         77
7069       work         75

Top 10 Words in Spam Messages:
              word  frequency
1265            2e       1856
12356        money        973
1096            2c        923
3374       account        866
5037          bank        794
8865          fund        710
17315  transaction        544
5554      business        511
6549       country        485
17330     transfer        419


## Extra features

In [174]:
#Create a new colum with preprocessed text
data['preprocessed_text'] = data['lemmatized_text']
#Do the spliting of the data for train and validation
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)


In [175]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,cleaned_text,no_stopwords_text,lemmatized_text,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...","----------- REGARDS, MR NELSON SMITH.KINDLY RE...","-- -- -- -- -- - REGARDS , MR NELSON SMITH.KIN...","-- -- -- -- -- - REGARDS , MR NELSON SMITH.KIN...",1,0,116
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...,I able reach oscar am. We supposed send pdb 11...,I able reach oscar am . We supposed send pdb 1...,I able reach oscar am . We supposed send pdb 1...,1,0,68
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...,; Huma Abedin B6I'm checking Pat 50k work Jack...,; Huma Abedin B6I 'm checking Pat 50k work Jac...,; Huma Abedin B6I 'm checking Pat 50k work Jac...,1,0,91
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...,I announced Monday - can't today,I announced Monday - ca n't today,I announced Monday - ca n't today,1,0,33
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...,1,1,1322


## How would work the Bag of Words with Count Vectorizer concept?

In [176]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Example dataset (Replace with your actual dataset)
data = pd.DataFrame({
    'preprocessed_text': [
        "Congratulations! You've won a $1000 gift card. Click here to claim.",
        "Meeting at 5pm today.",
        "Get rich quickly by investing in this one-time offer.",
        "Lunch at 12? Let me know if you're free.",
        "You’ve been selected for a special prize! Don’t miss out!"
    ],
    'label': ['spam', 'ham', 'spam', 'ham', 'spam']
})

# Step 1: Define money symbols and suspicious words
money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

# Step 2: Add 'money_mark' feature (1 if money symbol present, 0 otherwise)
data['money_mark'] = data['preprocessed_text'].str.contains(money_simbol_list, case=False, regex=True) * 1

# Step 3: Add 'suspicious_words' feature (1 if suspicious word present, 0 otherwise)
data['suspicious_words'] = data['preprocessed_text'].str.contains(suspicious_words, case=False, regex=True) * 1

# Step 4: Add 'text_len' feature (length of the text)
data['text_len'] = data['preprocessed_text'].apply(len)

# Step 5: Vectorize the text using CountVectorizer (Bag of Words model)
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 1))  # Use unigrams
X_bag_of_words = vectorizer.fit_transform(data['preprocessed_text'])

# Step 6: Combine Bag of Words with extra features (money_mark, suspicious_words, text_len)
extra_features = data[['money_mark', 'suspicious_words', 'text_len']].values
combined_features = np.hstack((X_bag_of_words.toarray(), extra_features))  # Combine BoW and extra features

# Step 7: Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, data['label'], test_size=0.3, random_state=42)

# Step 8: Train the Multinomial Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Step 9: Predict and evaluate the model
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Output results
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


Accuracy: 0.50
Classification Report:
              precision    recall  f1-score   support

         ham       0.00      0.00      0.00         1
        spam       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



c:\Users\diego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [189]:
# Step 1: Re-load the test data (since it may not be in memory)
test_data = pd.read_csv('../data/kg_test.csv')

# Step 2: Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])

# Step 3: Predict using the trained model
test_predictions = model.predict(X_test_tfidf)

# Step 4: Output the predictions to a CSV file
output = pd.DataFrame({'text': test_data['text'], 'prediction': test_predictions})
output.to_csv('../data/test_predictions.csv', index=False)

print("Predictions have been saved to test_predictions.csv")


Predictions have been saved to test_predictions.csv


In [190]:
# Importing necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
train_data = pd.read_csv('../data/kg_train.csv')
# Step 1: Data preparation
X = train_data['text']
y = train_data['label']

# Step 2: Train-test split (if not already done)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Initializing TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Step 4: Fitting TF-IDF on training data and transforming it
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Step 5: Transforming validation data
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Now X_train_tfidf and X_val_tfidf are ready for model training


In [191]:
# Import necessary libraries for model training
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Step 6: Initialize Logistic Regression model
model = LogisticRegression()

# Step 7: Fit the model on the TF-IDF transformed training data
model.fit(X_train_tfidf, y_train)

# Step 8: Predict on the validation set
y_pred = model.predict(X_val_tfidf)

# Step 9: Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)

# Output the results
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


Accuracy: 0.9731768650461022
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       674
           1       1.00      0.94      0.97       519

    accuracy                           0.97      1193
   macro avg       0.98      0.97      0.97      1193
weighted avg       0.97      0.97      0.97      1193



In [193]:
# Step 10: Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])

# Step 11: Predict using the trained model
test_predictions = model.predict(X_test_tfidf)

# Step 12: Output the predictions (e.g., to a CSV file)
output = pd.DataFrame({'text': test_data['text'], 'prediction': test_predictions})
output.to_csv('../data/test_predictions.csv', index=False)

print("Predictions have been saved to test_predictions.csv")


Predictions have been saved to test_predictions.csv


## And the Train a Classifier?

In [196]:
# Your code
# Step 1: Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Step 2: Prepare data (if not already done)
X = train_data['text']
y = train_data['label']

# Step 3: Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Initialize TF-IDF Vectorizer and fit on the training data
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Step 5: Initialize the Logistic Regression model
model = LogisticRegression()

# Step 6: Train the classifier on the training data
model.fit(X_train_tfidf, y_train)

# Step 7: Predict on the validation data
y_pred = model.predict(X_val_tfidf)

# Step 8: Evaluate the model's performance
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)

# Output the evaluation results
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


Accuracy: 0.9731768650461022
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       674
           1       1.00      0.94      0.97       519

    accuracy                           0.97      1193
   macro avg       0.98      0.97      0.97      1193
weighted avg       0.97      0.97      0.97      1193



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [84]:
# Your code